In [1]:
import os
import pandas as pd
import json

os.chdir("../CodeBase")
from PoloniexAPI import *

with open("../Secrets/secrets.json") as f:
    secrets = json.load(f)
    
api = poloniex(secrets['poloniex']['key'], secrets['poloniex']['secret'])
history = pd.read_csv("../Data/TradingHistory/PoloniexHistory.csv")
history.head()

,Date,Market,Category,Type,Price,Amount,Total,Fee,Order Number,Base Total Less Fee,Quote Total Less Fee
0,2018-06-07 22:17:16,DCR/BTC,Exchange,Sell,0.013907,1.119300,0.015566,0.1%,45094208814,0.015551,-1.119300
1,2018-05-30 16:36:03,GNT/ETH,Exchange,Sell,0.001055,150.741947,0.159033,0.1%,19696726743,0.158874,-150.741947
2,2018-05-29 15:03:21,LTC/XMR,Exchange,Sell,0.755000,0.033209,0.025073,0.1%,47810692586,0.025048,-0.033209
3,2018-05-29 15:03:18,LTC/XMR,Exchange,Sell,0.755000,0.920433,0.694927,0.1%,47810692586,0.694232,-0.920433
4,2018-05-25 05:45:44,STR/BTC,Exchange,Buy,0.000039,255.558395,0.010000,0.2%,77420584007,-0.010000,255.047278


In [2]:
tax = history[["Date", "Order Number", "Market", "Type", "Price", "Amount", "Total", "Fee", "Base Total Less Fee"]]
tax["BaseCurrency"] = tax.Market.str.split("/").str[1]
tax.head()

/home/nburns/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Date,Order Number,Market,Type,Price,Amount,Total,Fee,Base Total Less Fee,BaseCurrency
0,2018-06-07 22:17:16,45094208814,DCR/BTC,Sell,0.013907,1.119300,0.015566,0.1%,0.015551,BTC
1,2018-05-30 16:36:03,19696726743,GNT/ETH,Sell,0.001055,150.741947,0.159033,0.1%,0.158874,ETH
2,2018-05-29 15:03:21,47810692586,LTC/XMR,Sell,0.755000,0.033209,0.025073,0.1%,0.025048,XMR
3,2018-05-29 15:03:18,47810692586,LTC/XMR,Sell,0.755000,0.920433,0.694927,0.1%,0.694232,XMR
4,2018-05-25 05:45:44,77420584007,STR/BTC,Buy,0.000039,255.558395,0.010000,0.2%,-0.010000,BTC


In [3]:
tax.BaseCurrency.unique()

array(['BTC', 'ETH', 'XMR', 'USDT'], dtype=object)

In [4]:
currency_history = {} 
for base in tax.BaseCurrency.unique():
    if (base != "USDT"):
        x = "USDT_" + base
        chart_data = pd.DataFrame.from_dict(api.returnChartData(x, "2017-04-01 00:00:00", "2018-04-01 00:00:00", 300))
        chart_data['date'] = pd.to_datetime(chart_data['date'], unit="s")
        chart_data['Currency'] = x
        chart_data['BaseCurrency'] = base
        currency_history[x] = chart_data[["date", "close", "Currency", "BaseCurrency"]]


/home/nburns/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/nburns/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/nburns/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [5]:
currency_history['USDT_BTC'].head()

,date,close,Currency,BaseCurrency
0,2017-03-31 11:00:00,1046.501244,USDT_BTC,BTC
1,2017-03-31 11:05:00,1046.501244,USDT_BTC,BTC
2,2017-03-31 11:10:00,1046.016568,USDT_BTC,BTC
3,2017-03-31 11:15:00,1044.980000,USDT_BTC,BTC
4,2017-03-31 11:20:00,1047.000000,USDT_BTC,BTC


For each transaction, I will round the transaction timestamp up to the nearest 5 minutes, then join to the currency_history data to get the dollar value in USDT. 

In [6]:
import datetime

def round_to_5min(t):
    delta = datetime.timedelta(minutes=t.minute%5, 
                               seconds=t.second, 
                               microseconds=t.microsecond)
    t -= delta
    if delta > datetime.timedelta(0):
        t += datetime.timedelta(minutes=5)
    return t

tax['TransactionWindow'] = pd.to_datetime(tax['Date']).map(round_to_5min)

In [7]:
taxes = tax.loc[(tax['Date'] >= "2017-04-01 00:00:00") & (tax['Date'] <= "2018-03-31 23:59:59")]
taxes.head()

,Date,Order Number,Market,Type,Price,Amount,Total,Fee,Base Total Less Fee,BaseCurrency,TransactionWindow
971,2018-03-12 18:18:15,250099613151,XMR/BTC,Buy,0.027890,0.850,0.023706,0.15%,-0.023706,BTC,2018-03-12 18:20:00
972,2018-03-02 04:41:14,246158902806,XMR/BTC,Sell,0.029496,0.775,0.022859,0.15%,0.022825,BTC,2018-03-02 04:45:00
973,2018-02-28 04:41:22,156831165896,LTC/BTC,Buy,0.020060,1.000,0.020060,0.15%,-0.020060,BTC,2018-02-28 04:45:00
974,2018-02-27 11:04:28,126168616206,XRP/BTC,Buy,0.000088,73.000,0.006409,0.15%,-0.006409,BTC,2018-02-27 11:05:00
975,2018-02-27 07:23:56,25040626333,DGB/BTC,Sell,0.000004,750.000,0.002737,0.15%,0.002733,BTC,2018-02-27 07:25:00


In [8]:
def get_usdt(base_, window_, amount_):
    
    if base_ == "USDT":
        return ["USDT_USDT", 1, amount_]
    
    pair = "USDT_" + base_
    history =  currency_history[pair].loc[currency_history[pair]['date'] == window_]        
    converted = amount_ * history.iloc[0]["close"]
    return [history.iloc[0]["Currency"], history.iloc[0]["close"], converted]

taxes["USDT_Market"], taxes["MarketClose"], taxes["Total_USDT"] = 0, 0, 0

/home/nburns/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [9]:
markets = []
close = []
total = []
for x in range(taxes.shape[0]):
    a, b, c = get_usdt(taxes.iloc[x]["BaseCurrency"],
                       taxes.iloc[x]["TransactionWindow"],
                       taxes.iloc[x]["Base Total Less Fee"])
    markets.append(a)
    close.append(b)
    total.append(c)



In [10]:
taxes["USDT_Market"] = markets
taxes["MarketClose"] = close
taxes["Total_USDT"] = total

/home/nburns/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/nburns/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/nburns/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [11]:
taxes.iloc[:20]

,Date,Order Number,Market,Type,Price,Amount,Total,Fee,Base Total Less Fee,BaseCurrency,TransactionWindow,USDT_Market,MarketClose,Total_USDT
971,2018-03-12 18:18:15,250099613151,XMR/BTC,Buy,0.027890,0.850000,0.023706,0.15%,-0.023706,BTC,2018-03-12 18:20:00,USDT_BTC,9235.120000,-218.932372
972,2018-03-02 04:41:14,246158902806,XMR/BTC,Sell,0.029496,0.775000,0.022859,0.15%,0.022825,BTC,2018-03-02 04:45:00,USDT_BTC,10970.106692,250.394002
973,2018-02-28 04:41:22,156831165896,LTC/BTC,Buy,0.020060,1.000000,0.020060,0.15%,-0.020060,BTC,2018-02-28 04:45:00,USDT_BTC,10934.920899,-219.354513
974,2018-02-27 11:04:28,126168616206,XRP/BTC,Buy,0.000088,73.000000,0.006409,0.15%,-0.006409,BTC,2018-02-27 11:05:00,USDT_BTC,10688.758960,-68.508532
975,2018-02-27 07:23:56,25040626333,DGB/BTC,Sell,0.000004,750.000000,0.002737,0.15%,0.002733,BTC,2018-02-27 07:25:00,USDT_BTC,10334.873000,28.249342
976,2018-02-26 11:49:52,48483959620,XEM/BTC,Buy,0.000039,258.000000,0.010088,0.15%,-0.010088,BTC,2018-02-26 11:50:00,USDT_BTC,9722.340570,-98.077027
977,2018-02-23 23:46:25,85195450396,XRP/USDT,Buy,0.947477,345.000000,326.879700,0.25%,-326.879700,USDT,2018-02-23 23:50:00,USDT_USDT,1.000000,-326.879700
978,2018-02-23 23:42:10,85194210637,XRP/USDT,Buy,0.940019,10.000000,9.400194,0.25%,-9.400194,USDT,2018-02-23 23:45:00,USDT_USDT,1.000000,-9.400194
979,2018-02-23 23:40:33,126973622394,XRP/BTC,Buy,0.000093,72.024369,0.006679,0.25%,-0.006679,BTC,2018-02-23 23:45:00,USDT_BTC,10182.272705,-68.005465
980,2018-02-23 23:40:33,126973622394,XRP/BTC,Buy,0.000093,27.975631,0.002594,0.25%,-0.002594,BTC,2018-02-23 23:45:00,USDT_BTC,10182.272705,-26.414648


In [14]:
taxes.to_csv("../Data/PoloniexTradingHistory_USDTConverted.csv", index=False)